In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import DataLoader, TensorDataset

# Load and preprocess data
file_path = '/content/data.xlsx'  # Update with your file path
data = pd.read_excel(file_path)

# Normalize SPI, VCI, and TCI
scaler = MinMaxScaler()
data[['SPI', 'VCI', 'TCI']] = scaler.fit_transform(data[['SPI', 'VCI', 'TCI']])

# Create sequences
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
    return np.array(sequences)

data['location'] = list(zip(data['X'], data['Y']))
data_pivoted = data.pivot_table(index=['Year', 'Month'], columns='location', values=['SPI', 'VCI', 'TCI'])
data_pivoted = data_pivoted.fillna(method='ffill').fillna(method='bfill')

sequence_length = 12
spi_sequences = create_sequences(data_pivoted['SPI'].values, sequence_length)
vci_sequences = create_sequences(data_pivoted['VCI'].values, sequence_length)
tci_sequences = create_sequences(data_pivoted['TCI'].values, sequence_length)
combined_sequences = np.stack((spi_sequences, vci_sequences, tci_sequences), axis=-1)

split_ratio = 0.8
split_index = int(len(combined_sequences) * split_ratio)
train_sequences = combined_sequences[:split_index]
test_sequences = combined_sequences[split_index:]

# Convert sequences to PyTorch tensors
train_sequences_tensor = torch.tensor(train_sequences, dtype=torch.float32)
test_sequences_tensor = torch.tensor(test_sequences, dtype=torch.float32)

train_loader = DataLoader(TensorDataset(train_sequences_tensor[:, :-1], train_sequences_tensor[:, -1]), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(test_sequences_tensor[:, :-1], test_sequences_tensor[:, -1]), batch_size=32, shuffle=False)


In [2]:
import torch.nn as nn
import torch.optim as optim

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size):
        super(ConvLSTMCell, self).__init__()
        padding = kernel_size // 2  # Padding to keep input-output spatial dimensions same
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding = padding

        self.conv = nn.Conv2d(in_channels=input_dim + hidden_dim,
                              out_channels=4 * hidden_dim,
                              kernel_size=kernel_size,
                              padding=padding)

    def forward(self, x, h, c):
        combined = torch.cat([x, h], dim=1)  # concatenate along channel axis
        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)
        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

    def init_hidden(self, batch_size, shape):
        h = torch.zeros(batch_size, self.hidden_dim, shape[0], shape[1], device=self.conv.weight.device)
        c = torch.zeros(batch_size, self.hidden_dim, shape[0], shape[1], device=self.conv.weight.device)
        return h, c


class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers, output_dim):
        super(ConvLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        self.cells = nn.ModuleList([ConvLSTMCell(input_dim if i == 0 else hidden_dim,
                                                 hidden_dim,
                                                 kernel_size) for i in range(num_layers)])
        self.conv = nn.Conv2d(hidden_dim, output_dim, kernel_size=1)  # Final convolution to match output_dim

    def forward(self, x):
        batch_size, seq_len, num_nodes, num_features = x.size()
        x = x.view(batch_size * num_nodes, seq_len, num_features, 1, 1)  # Reshape for ConvLSTM
        h, c = self.init_hidden(batch_size * num_nodes, (1, 1))

        for t in range(seq_len):
            x_t = x[:, t, :, :, :]  # Shape: (batch_size * num_nodes, num_features, 1, 1)
            h, c = self.cells[0](x_t, h, c)

        for layer in range(1, self.num_layers):
            h, c = self.cells[layer](h, h, c)

        output = self.conv(h)  # Shape: (batch_size * num_nodes, output_dim, 1, 1)
        output = output.view(batch_size, num_nodes, self.output_dim)
        return output

    def init_hidden(self, batch_size, shape):
        return self.cells[0].init_hidden(batch_size, shape)


input_dim = 3
hidden_dim = 64
output_dim = 3
kernel_size = 3
num_layers = 1

model = ConvLSTM(input_dim, hidden_dim, kernel_size, num_layers, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [3]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}')


Epoch 1/100, Loss: 0.5376
Epoch 2/100, Loss: 0.4972
Epoch 3/100, Loss: 0.4574
Epoch 4/100, Loss: 0.4075
Epoch 5/100, Loss: 0.3285
Epoch 6/100, Loss: 0.2060
Epoch 7/100, Loss: 0.0640
Epoch 8/100, Loss: 0.0873
Epoch 9/100, Loss: 0.0333
Epoch 10/100, Loss: 0.0432
Epoch 11/100, Loss: 0.0434
Epoch 12/100, Loss: 0.0329
Epoch 13/100, Loss: 0.0321
Epoch 14/100, Loss: 0.0310
Epoch 15/100, Loss: 0.0283
Epoch 16/100, Loss: 0.0294
Epoch 17/100, Loss: 0.0277
Epoch 18/100, Loss: 0.0273
Epoch 19/100, Loss: 0.0262
Epoch 20/100, Loss: 0.0272
Epoch 21/100, Loss: 0.0281
Epoch 22/100, Loss: 0.0270
Epoch 23/100, Loss: 0.0255
Epoch 24/100, Loss: 0.0264
Epoch 25/100, Loss: 0.0260
Epoch 26/100, Loss: 0.0269
Epoch 27/100, Loss: 0.0278
Epoch 28/100, Loss: 0.0261
Epoch 29/100, Loss: 0.0261
Epoch 30/100, Loss: 0.0265
Epoch 31/100, Loss: 0.0254
Epoch 32/100, Loss: 0.0257
Epoch 33/100, Loss: 0.0251
Epoch 34/100, Loss: 0.0269
Epoch 35/100, Loss: 0.0261
Epoch 36/100, Loss: 0.0260
Epoch 37/100, Loss: 0.0260
Epoch 38/1

In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Function to flatten the 3D arrays into 2D arrays
def flatten_predictions(predictions, targets):
    num_batches, num_nodes, num_features = predictions.shape
    predictions_flat = predictions.reshape(num_batches * num_nodes, num_features)
    targets_flat = targets.reshape(num_batches * num_nodes, num_features)
    return predictions_flat, targets_flat

# Evaluation with scaled scores
model.eval()
train_predictions = []
train_targets = []
test_predictions = []
test_targets = []

# Collect predictions and targets for train set
with torch.no_grad():
    for inputs, targets in train_loader:
        outputs = model(inputs)
        train_predictions.append(outputs.numpy())
        train_targets.append(targets.numpy())

# Collect predictions and targets for test set
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        test_predictions.append(outputs.numpy())
        test_targets.append(targets.numpy())

# Concatenate all predictions and targets
train_predictions = np.concatenate(train_predictions, axis=0)
train_targets = np.concatenate(train_targets, axis=0)
test_predictions = np.concatenate(test_predictions, axis=0)
test_targets = np.concatenate(test_targets, axis=0)

# Flatten the predictions and targets
train_predictions_flat, train_targets_flat = flatten_predictions(train_predictions, train_targets)
test_predictions_flat, test_targets_flat = flatten_predictions(test_predictions, test_targets)

# Calculate evaluation metrics for train set
train_mae = mean_absolute_error(train_targets_flat, train_predictions_flat)
train_rmse = np.sqrt(mean_squared_error(train_targets_flat, train_predictions_flat))
train_r2 = r2_score(train_targets_flat, train_predictions_flat)

# Calculate evaluation metrics for test set
test_mae = mean_absolute_error(test_targets_flat, test_predictions_flat)
test_rmse = np.sqrt(mean_squared_error(test_targets_flat, test_predictions_flat))
test_r2 = r2_score(test_targets_flat, test_predictions_flat)

print(f'Train MAE: {train_mae:.4f}, RMSE: {train_rmse:.4f}, R2: {train_r2:.4f}')
print(f'Test MAE: {test_mae:.4f}, RMSE: {test_rmse:.4f}, R2: {test_r2:.4f}')

# Unscale predictions and targets
train_predictions_unscaled = scaler.inverse_transform(train_predictions_flat)
train_targets_unscaled = scaler.inverse_transform(train_targets_flat)
test_predictions_unscaled = scaler.inverse_transform(test_predictions_flat)
test_targets_unscaled = scaler.inverse_transform(test_targets_flat)

# Calculate unscaled RMSE and MAE
train_rmse_unscaled = np.sqrt(mean_squared_error(train_targets_unscaled, train_predictions_unscaled))
train_mae_unscaled = mean_absolute_error(train_targets_unscaled, train_predictions_unscaled)
test_rmse_unscaled = np.sqrt(mean_squared_error(test_targets_unscaled, test_predictions_unscaled))
test_mae_unscaled = mean_absolute_error(test_targets_unscaled, test_predictions_unscaled)

print(f'Unscaled Train RMSE: {train_rmse_unscaled:.4f}, MAE: {train_mae_unscaled:.4f}')
print(f'Unscaled Test RMSE: {test_rmse_unscaled:.4f}, MAE: {test_mae_unscaled:.4f}')


Train MAE: 0.0989, RMSE: 0.1589, R2: -0.1203
Test MAE: 0.0984, RMSE: 0.1555, R2: -0.0736
Unscaled Train RMSE: 203.5573, MAE: 51.8917
Unscaled Test RMSE: 443.9520, MAE: 94.6520
